In [7]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')

In [8]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [9]:
!pip install cohere langchain openai tiktoken -q

In [10]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2023-12-30 13:14:00--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10203 (10.0K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]   9.96K  --.-KB/s    in 0s      

2023-12-30 13:14:01 (43.5 MB/s) - ‘utils.py.1’ saved [10203/10203]



In [11]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI
import passwordfile
#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = passwordfile.course_api_key

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [12]:
df.head()

,text_id,text
0,9,#вакансия #vacancy #Python #удаленка #flask #r...
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...
2,28,#вакансия #vacancy #job #senior #data #DB #dat...
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...
4,18,#vacancy #job #analyst #travel #sirenatravel #...


In [15]:
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts import ChatPromptTemplate

In [17]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [35]:
job_title_schema = ResponseSchema(name="job_title",
                             description="Извлеките название должности на основе предоставленного текста. НАЗВАНИЕ ГРЕЙДА НЕ ДОЛЖНО СОДЕРЖАТЬСЯ В НАЗВАНИИ ДОЛЖНОСТИ. ЕГО НУЖНО УБРАТЬ. НАПРИМЕР Senior Python developer -> Python Developer Если информации нет, ответь None")

company_schema = ResponseSchema(name="company",
                                      description="Извлеките название компании на основе предоставленного текста. Пишется только название компании, (не пишем финтех, крупная компания, мобильная игра и т.д.).Если эта информация не найдена, выведи None.")

salary_schema = ResponseSchema(name="salary",
                                    description="Извлеките информацию о зарплате из текста, учитывая формат: числовое значение без пробелов, с указанием валюты (руб. или $) и, при необходимости, указанием отдельно нижней или верхней границы, если это указано в тексте. Если дана только нижняя граница, то нужно писать 'от n руб.' или 'от n $'. Если дана только верхняя граница, то нужно писать 'до n руб.' или 'до n $'. Если дан диапазон, то ответ обязательно должен выглядеть как 'x-y руб.' или 'x-y $'. Если просто найдено число, то пишешь 'n руб' или 'n $'. Если эта информация не найдена, выведи None.")

tg_schema = ResponseSchema(name="tg",
                                      description="Извлеките контактные данные для связи из текста, предполагая, что они указаны в формате @username. Если эта информация не найдена, выведи None.")

grade_schema = ResponseSchema(name="grade",
                                      description="Извлеките уровень грейда (если присутствует) из текста, включая значения intern, junior, junior+, middle, middle+, senior, lead. Если представлены несколько грейдов, выведи их через запятую в порядке возрастания. Если эта информация не найдена, выведи None.")

response_schemas = [job_title_schema,
                    company_schema,
                    salary_schema,
                    tg_schema,
                    grade_schema]

In [36]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [37]:
format_instructions = output_parser.get_format_instructions()

In [34]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"job_title": string  // Извлеките название должности на основе предоставленного текста. Название должности не должно включать в себя грейд. Его нужно убрать. Если информации нет, ответь None
	"company": string  // Извлеките название компании на основе предоставленного текста. Пишется только название компании, (не пишем финтех, крупная компания, мобильная игра и т.д.).Если эта информация не найдена, выведи None.
	"salary": string  // Извлеките информацию о зарплате из текста, учитывая формат: числовое значение без пробелов, с указанием валюты (руб. или $) и, при необходимости, указанием отдельно нижней или верхней границы, если это указано в тексте. Если дана только нижняя граница, то нужно писать 'от n руб.' или 'от n $'. Если дана только верхняя граница, то нужно писать 'до n руб.' или 'до n $'. Если дан диапазон, то ответ обязательно долж

In [38]:
prompt_template = """
Представь что ты мой личный карьерный ассистент и тебе нужно извлекать информацию из текстов вакансий.
Получите следующую информацию из текста вакансии:

Позиция (job_title).
Компания (company).
Зарплата (salary).
Контакты для связи (tg).
Грейд (grade).

Используя предоставленное описание вакансии, извлеките следующую информацию строго соблюдая следующие инструкции:

{format_instructions}

ПОЖАЛУЙСТА, обратите внимание на следующие условия:
- Все значения должны соответствовать условиям, указанным в запросе, включая форматирование и дополнительные указания по каждой колонке.
- Если какая-либо информация отсутствует в тексте вакансии, укажите значение None.

text: {text}
"""
prompt = ChatPromptTemplate.from_template(template=prompt_template)


In [39]:
messages = prompt.format_messages(text=df.iloc[0]['text'],
                                format_instructions=format_instructions)

response = llm(messages)
print(response.content)

```json
{
	"job_title": "Python Developer",
	"company": "Collectly",
	"salary": "6000-9000 $",
	"tg": "@ann_gfio",
	"grade": "senior"
}
```


In [40]:
output_dict = output_parser.parse(response.content)

In [41]:
output_dict

{'job_title': 'Python Developer',
 'company': 'Collectly',
 'salary': '6000-9000 $',
 'tg': '@ann_gfio',
 'grade': 'senior'}

In [42]:
answers = {'job_title': [],
 'company': [],
 'salary': [],
 'tg': [],
 'grade': []}
for index, row in df.iterrows():
  print('Index', index, end=' ')
  text = row["text"]
  try:
    messages = prompt.format_messages(text=text,
                                format_instructions=format_instructions)
    response = llm(messages)
    output_dict = output_parser.parse(response.content)
  except:
    output_dict = {'job_title': None,
 'company': None,
 'salary': None,
 'tg': None,
 'grade': None}
  print(output_dict)
  answers['job_title'].append(output_dict['job_title'])
  answers['company'].append(output_dict['company'])
  answers['salary'].append(output_dict['salary'])
  answers['tg'].append(output_dict['tg'])
  answers['grade'].append(output_dict['grade'])
print(answers)

Index 0 {'job_title': 'Python Developer', 'company': 'Collectly', 'salary': '6000-9000 $', 'tg': '@ann_gfio', 'grade': 'senior'}
Index 1 {'job_title': 'Системный аналитик', 'company': 'АКЦИОНЕРНОЕ ОБЩЕСТВО «ГЛАВНЫЙ НАУЧНЫЙ ИННОВАЦИОННЫЙ ВНЕДРЕНЧЕСКИЙ ЦЕНТР»', 'salary': '320 000 - 370 000 руб.', 'tg': '@NatalyaMaki', 'grade': None}
Index 2 {'job_title': 'Database Administrator', 'company': 'Match Systems', 'salary': 'от 3000 $', 'tg': '@lex_kertis', 'grade': 'Senior'}
Index 3 {'job_title': 'Руководитель отдела системного администрирования', 'company': 'финтех компания', 'salary': 'от 3000 $', 'tg': '@ResearcherRIT', 'grade': None}
Index 4 {'job_title': 'Analyst', 'company': 'Sirena Travel', 'salary': '120-180к. net', 'tg': '@ann_gfio', 'grade': None}
Index 5 {'job_title': 'DevOps инженер', 'company': 'Mad Devs', 'salary': 'до 5000 $', 'tg': '@recruiter_maddevs', 'grade': 'Senior'}
Index 6 {'job_title': 'Системный аналитик', 'company': 'Платформа', 'salary': '180 000 - 300 000 руб.', 'tg

In [44]:
for naz, lis in answers.items():
  df[naz] = lis

In [45]:
df.head()

,text_id,text,job_title,company,salary,tg,grade
0,9,#вакансия #vacancy #Python #удаленка #flask #r...,Python Developer,Collectly,6000-9000 $,@ann_gfio,senior
1,31,#ВАКАНСИЯ #Системный_аналитик #РФ\n \nАКЦИОНЕ...,Системный аналитик,АКЦИОНЕРНОЕ ОБЩЕСТВО «ГЛАВНЫЙ НАУЧНЫЙ ИННОВАЦИ...,320 000 - 370 000 руб.,@NatalyaMaki,None
2,28,#вакансия #vacancy #job #senior #data #DB #dat...,Database Administrator,Match Systems,от 3000 $,@lex_kertis,Senior
3,49,#вакансия #fulltime #remote \n\n🔎 Ищем Руковод...,Руководитель отдела системного администрирования,финтех компания,от 3000 $,@ResearcherRIT,None
4,18,#vacancy #job #analyst #travel #sirenatravel #...,Analyst,Sirena Travel,120-180к. net,@ann_gfio,None


In [46]:
df.to_csv("submission_50_rows", index=False)